# skilledagent

> Agent Building Block that uses skills

In [ ]:
#| default_exp skilledagent


# SkilledAgent

> Extend claudette/lisette agents with Agent Skills for enhanced capabilities

## Overview

`SkilledAgent` wraps a claudette/lisette agent and extends it with the ability to discover, activate, and use Agent Skills. Skills provide three types of enhancements:

1. **Procedural knowledge** - Instructions injected into system prompt
2. **Tools** - Python functions registered with the agent's tool loop
3. **Resources** - References, assets, and scripts the agent can access

## Core Concepts

### Skill Activation Flow

1. Agent discovers available skills via `discover_all()`
2. User/agent activates skill: `agent.activate_skill('skill-name')`
3. Skill validation:
   - Check skill's capabilities ⊆ agent's SecurityPolicy
   - Validate scripts if `verify_scripts=True`
4. Load procedural knowledge into system prompt
5. Register skill's tools with agent
6. Agent can now use skill in tool loop

### Security Model

Skills are validated against agent's SecurityPolicy:
- **Capabilities**: filesystem, network, subprocess, eval
- **Tools**: External bash commands (enforced via safe_run)
- **Verification**: Static analysis of scripts before execution

### Progressive Disclosure

Following Agent Skills spec:
- **Discovery**: Load only name + description
- **Activation**: Load full instructions
- **On-demand**: Load references/assets when needed

## Key Methods

- `SkilledAgent(agent, policy)` - Wrap existing agent with skill support
- `.activate_skill(name)` - Load and validate a skill
- `.list_skills()` - Show available skills
- `.teach(skill_name)` - Activate skill and explain to user

## Integration with claudette/lisette

Tools from skills are registered using the same mechanism as built-in tools:
- Type annotations define parameters
- Docstrings provide descriptions
- Agent decides when to call them in tool loop

In [ ]:
#| export
from claudette import *

In [ ]:
#| export
from skillhelper.core import *
from skillhelper.core import SecurityPolicy, validate_capabilities

In [ ]:
model = models[1]
model

'claude-sonnet-4-5'

In [ ]:
from dialoghelper import *
from dialoghelper.inspecttools import *
from fastcore.tools import *

In [ ]:
#| export
from fastcore import *
from fastcore.basics import AttrDict
import strictyaml
from fastcore.xtras import *
from fastcore.basics import first
from pathlib import Path
from dataclasses import dataclass
from typing import Optional
from fastcore.basics import patch
from safecmd import safe_run
import ast
import os

Tools available from dialoghelper: &`[curr_dialog, msg_idx, add_html, find_msg_id, find_msgs, read_msg, del_msg, add_msg, update_msg, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines]`

Tools available from fastcore.tools: &`[rg, sed, view, create, insert, str_replace, strs_replace, replace_lines]`

In [ ]:
#| export
class SkilledAgent:
    "Extend claudette/lisette agent with Agent Skills"
    def __init__(self, agent, policy=None): self.agent,self.policy,self.active_skills = agent,policy,{}

In [ ]:
agent = Chat(model)
sa = SkilledAgent(agent)
sa.agent, sa.policy, sa.active_skills

(<claudette.core.Chat at 0x73b890b353a0>, None, {})

In [ ]:
discover_all()

[Skill(name='code-reviewer', description='Reviews code for style, bugs, and best practices. Use when asked to review or analyze code quality.', path=Path('../skills/code-reviewer'), instructions='# Code Reviewer Skill\n\nThis skill helps review code for common issues:\n\n1. Check for style violations\n2. Identify potential bugs\n3. Suggest improvements\n4. Verify best practices\n\n## Usage\n\nWhen activated, analyze the provided code and give constructive feedback on:\n- Code structure and organization\n- Potential bugs or edge cases\n- Performance considerations\n- Readability and maintainability\n\nSee references for detailed checklists.', license='MIT', compatibility='Works with any code analysis tools', allowed_tools='python', metadata={'version': '1.0.0', 'author': 'skillhelper-dev', 'tags': ['code', 'review', 'quality']}),
 Skill(name='hello-world', description='A minimal test skill that demonstrates the basic skill structure', path=Path('../skills/hello-world'), instructions='# 

In [ ]:
#| export
@patch
def list_skills(self:SkilledAgent): return discover_all()

In [ ]:
Skill.__dataclass_fields__.keys()

dict_keys(['name', 'description', 'path', 'instructions', 'license', 'compatibility', 'allowed_tools', 'metadata'])

In [ ]:
skill = first(discover_all())
dir(skill)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'allowed_tools',
 'asset_path',
 'compatibility',
 'description',
 'get_tools',
 'instructions',
 'license',
 'load_reference',
 'metadata',
 'name',
 'path',
 'run_script',
 'to_prompt']

In [ ]:
skill.get_tools()


{'check_style': <function code-reviewer.tools.check_style(code: str) -> str>,
 'count_lines': <function code-reviewer.tools.count_lines(filepath: str) -> dict>}

In [ ]:
#| export
@patch
def activate_skill(self:SkilledAgent, name:str):
    skill = first(s for s in discover_all() if s.name==name)
    if not skill: return AttrDict(success=False, skill=None, error=f"Skill '{name}' not found")
    
    # Security validation
    if self.policy:
        try: validate_capabilities(skill, self.policy)
        except ValueError as e: return AttrDict(success=False, skill=None, error=str(e))
    
    self.active_skills[name] = skill
    for t in skill.get_tools().values(): self.agent.tools.append(t)
    self.agent.h.append({'role': 'user', 'content': f"[Skill activated: {name}]\n\n{skill.instructions}"})
    return AttrDict(success=True, skill=skill, error=None)

In [ ]:
agent = Chat(model, tools=[])
sa = SkilledAgent(agent)
sa.activate_skill('code-reviewer')
sa.agent.tools

[<function code-reviewer.tools.check_style(code: str) -> str>,
 <function code-reviewer.tools.count_lines(filepath: str) -> dict>]

In [ ]:
sa.agent("Please check the style of this code: `def foo( x ):return x+1`")

[ToolUseBlock(id='toolu_01HjJmbVkcNjJcPj9wAkXAEw', input={'code': 'def foo( x ):return x+1'}, name='check_style', type='tool_use')]

<details>

- id: `msg_01WbZqmvvinF6mbtV7pLrXG6`
- content: `[{'id': 'toolu_01HjJmbVkcNjJcPj9wAkXAEw', 'input': {'code': 'def foo( x ):return x+1'}, 'name': 'check_style', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-5-20250929`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 960, 'output_tokens': 61, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
sa.agent()


While the automated style checker didn't flag any issues, let me provide a more detailed code review based on Python style best practices (PEP 8):

## Style Issues Found:

1. **Spacing around parameters**: `( x )` has unnecessary spaces inside parentheses
   - ❌ `def foo( x ):`
   - ✅ `def foo(x):`

2. **Missing space around operator**: `x+1` should have spaces around the `+` operator
   - ❌ `x+1`
   - ✅ `x + 1`

3. **Statement on same line**: The return statement should be on a new line
   - ❌ `):return x+1`
   - ✅ `):\n    return x + 1`

4. **Missing docstring**: Function lacks documentation

## Recommended Code:

```python
def foo(x):
    """Add 1 to the input value.
    
    Args:
        x: A numeric value
        
    Returns:
        The input value plus 1
    """
    return x + 1
```

## Summary:
- **Readability**: Low (cramped formatting)
- **PEP 8 Compliance**: Multiple violations
- **Functionality**: Correct, but needs better formatting

Would you like me to review any other aspects of this code?

<details>

- id: `msg_018khqEhdhndaRX9HqikqJre`
- content: `[{'citations': None, 'text': 'While the automated style checker didn\'t flag any issues, let me provide a more detailed code review based on Python style best practices (PEP 8):\n\n## Style Issues Found:\n\n1. **Spacing around parameters**: `( x )` has unnecessary spaces inside parentheses\n   - ❌ `def foo( x ):`\n   - ✅ `def foo(x):`\n\n2. **Missing space around operator**: `x+1` should have spaces around the `+` operator\n   - ❌ `x+1`\n   - ✅ `x + 1`\n\n3. **Statement on same line**: The return statement should be on a new line\n   - ❌ `):return x+1`\n   - ✅ `):\\n    return x + 1`\n\n4. **Missing docstring**: Function lacks documentation\n\n## Recommended Code:\n\n```python\ndef foo(x):\n    """Add 1 to the input value.\n    \n    Args:\n        x: A numeric value\n        \n    Returns:\n        The input value plus 1\n    """\n    return x + 1\n```\n\n## Summary:\n- **Readability**: Low (cramped formatting)\n- **PEP 8 Compliance**: Multiple violations\n- **Functionality**: Correct, but needs better formatting\n\nWould you like me to review any other aspects of this code?', 'type': 'text'}]`
- model: `claude-sonnet-4-5-20250929`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1038, 'output_tokens': 323, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
sa.agent.h

[{'role': 'user',
  'content': '[Skill activated: code-reviewer]\n\n# Code Reviewer Skill\n\nThis skill helps review code for common issues:\n\n1. Check for style violations\n2. Identify potential bugs\n3. Suggest improvements\n4. Verify best practices\n\n## Usage\n\nWhen activated, analyze the provided code and give constructive feedback on:\n- Code structure and organization\n- Potential bugs or edge cases\n- Performance considerations\n- Readability and maintainability\n\nSee references for detailed checklists.'},
 {'role': 'assistant',
  'content': [{'text': "Great! I'm ready to help review code. The code-reviewer skill is now activated, and I can assist you with:\n\n1. **Style violations** - checking code formatting and conventions\n2. **Potential bugs** - identifying logic errors and edge cases\n3. **Improvements** - suggesting better approaches and optimizations\n4. **Best practices** - verifying adherence to coding standards\n\nPlease provide the code you'd like me to review, a

We probably need to inject the instructuctions into the context somehow. Could you do some research on anthropics skill documentation about how best to do that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's what I found about how skills inject instructions:

**Key mechanism:** [*](https://leehanchung.github.io/blogs/2025/10/26/claude-skills-deep-dive/ "Claude Agent Skills: A First Principles Deep Dive") "When Claude invokes a skill, the system follows a simple workflow: it loads a markdown file (SKILL.md), expands it into detailed instructions, injects those instructions as new user messages into the conversation context."

**Progressive disclosure pattern:**
[*](https://markets.financialcontent.com/wral/article/tokenring-2025-12-24-anthropic-launches-agent-skills-open-standard-the-new-universal-language-for-ai-interoperability "Anthropic Launches “Agent Skills” Open Standard: The New Universal Language for AI Interoperability") "When an agent encounters a task, it navigates three levels of disclosure: first scanning metadata to see if a skill is relevant, then loading specific instructions, and finally accessing external scripts or resources only when execution is required."

**System prompt + dynamic loading:**
[*](https://www.anthropic.com/engineering/equipping-agents-for-the-real-world-with-agent-skills "Equipping agents for the real world with Agent Skills") "Skills are triggered in the context window via your system prompt... To start, the context window has the core system prompt and the metadata for each of the installed skills, along with the user's initial message."

[*](https://platform.claude.com/docs/en/agents-and-tools/agent-skills/overview "Agent Skills - Claude Docs") "When a Skill is triggered, Claude uses bash to read SKILL.md from the filesystem, bringing its instructions into the context window. If those instructions reference other files (like FORMS.md or a database schema), Claude reads those files too using additional bash commands."

**Two approaches emerge:**
1. **Metadata in system prompt** — skill name + description always present, so the agent knows what's available
2. **Instructions injected on activation** — as user messages when the skill is triggered

For your `SkilledAgent`, you'd want to:
- Add skill metadata summaries to the system prompt (so the agent knows what skills exist)
- When `activate_skill()` is called, inject the full `skill.instructions` into the conversation

Would you like to try adding the instructions to the system prompt, or injecting them as a user message?

## Prompting

**Key mechanism:** [*](https://leehanchung.github.io/blogs/2025/10/26/claude-skills-deep-dive/ "Claude Agent Skills: A First Principles Deep Dive") "When Claude invokes a skill, the system follows a simple workflow: it loads a markdown file (SKILL.md), expands it into detailed instructions, injects those instructions as new user messages into the conversation context."

**Progressive disclosure pattern:**
[*](https://markets.financialcontent.com/wral/article/tokenring-2025-12-24-anthropic-launches-agent-skills-open-standard-the-new-universal-language-for-ai-interoperability "Anthropic Launches “Agent Skills” Open Standard: The New Universal Language for AI Interoperability") "When an agent encounters a task, it navigates three levels of disclosure: first scanning metadata to see if a skill is relevant, then loading specific instructions, and finally accessing external scripts or resources only when execution is required."

**System prompt + dynamic loading:**
[*](https://www.anthropic.com/engineering/equipping-agents-for-the-real-world-with-agent-skills "Equipping agents for the real world with Agent Skills") "Skills are triggered in the context window via your system prompt... To start, the context window has the core system prompt and the metadata for each of the installed skills, along with the user's initial message."

[*](https://platform.claude.com/docs/en/agents-and-tools/agent-skills/overview "Agent Skills - Claude Docs") "When a Skill is triggered, Claude uses bash to read SKILL.md from the filesystem, bringing its instructions into the context window. If those instructions reference other files (like FORMS.md or a database schema), Claude reads those files too using additional bash commands."

**Two approaches emerge:**
1. **Metadata in system prompt** — skill name + description always present, so the agent knows what's available
2. **Instructions injected on activation** — as user messages when the skill is triggered

For your `SkilledAgent`, you'd want to:
- Add skill metadata summaries to the system prompt (so the agent knows what skills exist)
- When `activate_skill()` is called, inject the full `skill.instructions` into the conversation

Would you like to try adding the instructions to the system prompt, or injecting them as a user message?

Could you do some research on the best prompt structure for the skills metadata?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

Here's what I found on the recommended format:

**System prompt format:**

[*](https://www.anthropic.com/engineering/equipping-agents-for-the-real-world-with-agent-skills "Equipping agents for the real world with Agent Skills") "This file must start with YAML frontmatter that contains some required metadata: name and description. At startup, the agent pre-loads the name and description of every installed skill into its system prompt."

[*](https://platform.claude.com/docs/en/agents-and-tools/agent-skills/overview "Agent Skills - Claude Docs") "The Skill's YAML frontmatter provides discovery information: `--- name: pdf-processing description: Extract text and tables from PDF files, fill forms, merge documents. Use when working with PDF files or when the user mentions PDFs, forms, or document extraction. ---` Claude loads this metadata at startup and includes it in the system prompt."

**Key constraints:**

[*](https://markets.financialcontent.com/wral/article/tokenring-2025-12-25-anthropic-unveils-agent-skills-open-standard-a-blueprint-for-modular-ai-autonomy "Anthropic Unveils 'Agent Skills' Open Standard: A Blueprint for ...") "To prevent the 'context window bloat' that often degrades the performance of models like Claude or GPT-4, the standard uses a three-tier loading system. Initially, only the skill's name and a brief 1,024-character description are loaded."

[*](https://venturebeat.com/ai/anthropic-launches-enterprise-agent-skills-and-opens-the-standard "Anthropic launches enterprise 'Agent Skills' and opens the ...") "Each skill takes only a few dozen tokens when summarized in the AI's context window, with full details loading only when the task requires them."

**Best practice for descriptions:**

[*](https://editorialge.com/anthropic-agent-skills-open-standard/ "Anthropic Agent Skills Open Standard Goes Public") "In practice, 'activation' relies heavily on how well a Skill is described. The clearer the description of 'when to use,' the easier it is for an agent to choose the right Skill without guesswork."

So for your system prompt, you'd want something like:

```
## Available Skills

- **code-reviewer**: Reviews code for style, bugs, and best practices. Use when asked to review or analyze code quality.
- **hello-world**: A minimal test skill that demonstrates the basic skill structure
```

Simple name + description list, with the description including "when to use" hints. Want to try implementing that?

In [ ]:
#| export
@patch
def _skills_summary(self:SkilledAgent):
    "Return formatted skills metadata for system prompt"
    skills = discover_all()
    if not skills: return ""
    lines = ["## Available Skills", ""]
    for s in skills:
        lines.append(f"- **{s.name}**: {s.description}")
    return "\n".join(lines)

In [ ]:
#| export
@patch
def update_sp(self:SkilledAgent):
    "Append skills summary to agent's system prompt"
    summary = self._skills_summary()
    if not summary: return
    base = self.agent.sp or ""
    self.agent.sp = f"{base}\n\n{summary}" if base else summary

In [ ]:
agent = Chat(model, sp="You are a helpful assistant.")
sa = SkilledAgent(agent)
sa.update_sp()
print(sa.agent.sp)

You are a helpful assistant.

## Available Skills

- **code-reviewer**: Reviews code for style, bugs, and best practices. Use when asked to review or analyze code quality.
- **hello-world**: A minimal test skill that demonstrates the basic skill structure


In [ ]:
agent = Chat(model, sp="You are a helpful assistant.", tools=[])
sa = SkilledAgent(agent)
sa.update_sp()
sa.activate_skill('code-reviewer')
sa.agent.h

[{'role': 'user',
  'content': '[Skill activated: code-reviewer]\n\n# Code Reviewer Skill\n\nThis skill helps review code for common issues:\n\n1. Check for style violations\n2. Identify potential bugs\n3. Suggest improvements\n4. Verify best practices\n\n## Usage\n\nWhen activated, analyze the provided code and give constructive feedback on:\n- Code structure and organization\n- Potential bugs or edge cases\n- Performance considerations\n- Readability and maintainability\n\nSee references for detailed checklists.'}]

In [ ]:
sa.agent("Review this code: `def add(a,b): return a+b`")

[ToolUseBlock(id='toolu_018kfQS9rBvTAcw1Rbhg4uF2', input={'code': 'def add(a,b): return a+b'}, name='check_style', type='tool_use')]

<details>

- id: `msg_01XCq4weaL1f6HAYtV3gFHMG`
- content: `[{'id': 'toolu_018kfQS9rBvTAcw1Rbhg4uF2', 'input': {'code': 'def add(a,b): return a+b'}, 'name': 'check_style', 'type': 'tool_use'}]`
- model: `claude-sonnet-4-5-20250929`
- role: `assistant`
- stop_reason: `tool_use`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1061, 'output_tokens': 63, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
sa.agent()

## Code Review: `add` Function

### ✅ Positives
- Simple and straightforward implementation
- Function name clearly describes its purpose
- No style violations detected

### ⚠️ Areas for Improvement

#### 1. **Spacing (PEP 8 Style)**
While no critical style issues were found, PEP 8 recommends spaces after commas:
```python
def add(a, b):  # Space after comma
    return a + b
```

#### 2. **Missing Type Hints**
Consider adding type hints for better code documentation and IDE support:
```python
def add(a: int, b: int) -> int:
    return a + b
```
Or for more flexibility:
```python
from typing import Union

def add(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:
    return a + b
```

#### 3. **Missing Docstring**
Add documentation to explain the function's purpose:
```python
def add(a, b):
    """
    Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        The sum of a and b
    """
    return a + b
```

#### 4. **No Input Validation**
The function will work with any types that support the `+` operator (numbers, strings, lists, etc.). Consider if this is intentional or if you need validation.

### 📝 Recommended Version
```python
def add(a: float, b: float) -> float:
    """
    Add two numbers together.
    
    Args:
        a: First number
        b: Second number
    
    Returns:
        The sum of a and b
    """
    return a + b
```

**Overall**: The code is functional but could benefit from better documentation and type hints for production use.

<details>

- id: `msg_012yUEnDRp4FANFpWXRS1VHt`
- content: `[{'citations': None, 'text': '## Code Review: `add` Function\n\n### ✅ Positives\n- Simple and straightforward implementation\n- Function name clearly describes its purpose\n- No style violations detected\n\n### ⚠️ Areas for Improvement\n\n#### 1. **Spacing (PEP 8 Style)**\nWhile no critical style issues were found, PEP 8 recommends spaces after commas:\n```python\ndef add(a, b):  # Space after comma\n    return a + b\n```\n\n#### 2. **Missing Type Hints**\nConsider adding type hints for better code documentation and IDE support:\n```python\ndef add(a: int, b: int) -> int:\n    return a + b\n```\nOr for more flexibility:\n```python\nfrom typing import Union\n\ndef add(a: Union[int, float], b: Union[int, float]) -> Union[int, float]:\n    return a + b\n```\n\n#### 3. **Missing Docstring**\nAdd documentation to explain the function\'s purpose:\n```python\ndef add(a, b):\n    """\n    Add two numbers together.\n    \n    Args:\n        a: First number\n        b: Second number\n    \n    Returns:\n        The sum of a and b\n    """\n    return a + b\n```\n\n#### 4. **No Input Validation**\nThe function will work with any types that support the `+` operator (numbers, strings, lists, etc.). Consider if this is intentional or if you need validation.\n\n### 📝 Recommended Version\n```python\ndef add(a: float, b: float) -> float:\n    """\n    Add two numbers together.\n    \n    Args:\n        a: First number\n        b: Second number\n    \n    Returns:\n        The sum of a and b\n    """\n    return a + b\n```\n\n**Overall**: The code is functional but could benefit from better documentation and type hints for production use.', 'type': 'text'}]`
- model: `claude-sonnet-4-5-20250929`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1141, 'output_tokens': 448, 'server_tool_use': None, 'service_tier': 'standard'}`

</details>

In [ ]:
result = sa.activate_skill('code-reviewer')
result.success, result.skill, result.error


(True,
 Skill(name='code-reviewer', description='Reviews code for style, bugs, and best practices. Use when asked to review or analyze code quality.', path=Path('../skills/code-reviewer'), instructions='# Code Reviewer Skill\n\nThis skill helps review code for common issues:\n\n1. Check for style violations\n2. Identify potential bugs\n3. Suggest improvements\n4. Verify best practices\n\n## Usage\n\nWhen activated, analyze the provided code and give constructive feedback on:\n- Code structure and organization\n- Potential bugs or edge cases\n- Performance considerations\n- Readability and maintainability\n\nSee references for detailed checklists.', license='MIT', compatibility='Works with any code analysis tools', allowed_tools='python', metadata={'version': '1.0.0', 'author': 'skillhelper-dev', 'tags': ['code', 'review', 'quality']}),
 None)

In [ ]:
policy = SecurityPolicy(allowed_tools={'python'}, allowed_capabilities={'filesystem'})
agent = Chat(model, sp="You are a helpful assistant.", tools=[])
sa = SkilledAgent(agent, policy=policy)
result = sa.activate_skill('code-reviewer')
result.success, result.error

(False,
 "Script fetch_data.py requires capabilities not in policy: {'network'}")

In [ ]:
policy = SecurityPolicy(allowed_tools={'python'}, allowed_capabilities={'filesystem'})
agent = Chat(model, sp="You are a helpful assistant.", tools=[])
sa = SkilledAgent(agent, policy=policy)
sa.update_sp()

# Try to activate and pass result to agent
result = sa.activate_skill('code-reviewer')
if not result.success:
    sa.agent(f"I tried to activate the code-reviewer skill but got this error: {result.error}")

In [ ]:
sa.agent.h[-1]


```python
{ 'content': [ { 'text': 'This error occurs because the `code-reviewer` '
                         "skill's `fetch_data.py` script is trying to use "
                         "network capabilities that aren't allowed by the "
                         'current security policy.\n'
                         '\n'
                         'Here are your options to resolve this:\n'
                         '\n'
                         '## 1. **Update the Policy File** (Recommended)\n'
                         'Add network capabilities to your policy '
                         'configuration:\n'
                         '\n'
                         '```yaml\n'
                         '# In your policy.yaml or security config\n'
                         'capabilities:\n'
                         '  network: true\n'
                         '  # or more restrictively:\n'
                         '  network:\n'
                         '    allowed_domains:\n'
                         '      - "api.github.com"\n'
                         '      - "raw.githubusercontent.com"\n'
                         '```\n'
                         '\n'
                         '## 2. **Modify the Skill**\n'
                         "If `fetch_data.py` doesn't actually need network "
                         'access, you can:\n'
                         '- Remove the network capability requirement from the '
                         "skill's manifest\n"
                         '- Modify the script to work with local data only\n'
                         '- Comment out or remove the network-dependent code\n'
                         '\n'
                         '## 3. **Use a Different Skill Version**\n'
                         "Check if there's a version of `code-reviewer` that "
                         "doesn't require network access, or use it without "
                         'the `fetch_data.py` component.\n'
                         '\n'
                         '## 4. **Run with Elevated Permissions** (Use '
                         'Cautiously)\n'
                         'If you trust the skill and need network access:\n'
                         '```bash\n'
                         '# Run with a permissive policy\n'
                         'your-command --policy permissive\n'
                         '```\n'
                         '\n'
                         '**Which approach would you like to take?** I can '
                         'help you implement any of these solutions. The most '
                         'common issue is that `fetch_data.py` might be '
                         'fetching coding standards or rules from a remote '
                         'source, which requires network permission.',
                 'type': 'text'}],
  'role': 'assistant'}
```